In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
import pickle
import datetime

import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Flatten,MaxPool2D,BatchNormalization,GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from keras.models import load_model

In [ ]:
ROOT_DIR = _YOUR_DIR_
TRAIN_PATH = os.path.join(ROOT_DIR, 'dataset/train')
TEST_PATH = os.path.join(ROOT_DIR, 'dataset/test')

# Image Data Generators

In [ ]:
batch_size = 32
img_height = 480
img_width = 480
target_size = (img_height, img_width)

In [ ]:

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical', 
        color_mode='rgb',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
    TRAIN_PATH, 
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical', 
    color_mode='rgb',
    subset='validation') 


test_generator = test_datagen.flow_from_directory(
        TEST_PATH,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical', 
        color_mode='rgb',
        shuffle=False)

Found 24104 images belonging to 3 classes.
Found 6026 images belonging to 3 classes.
Found 400 images belonging to 3 classes.


In [ ]:
counter = Counter(train_generator.classes)
counter1 = Counter(validation_generator.classes)
counter2 = Counter(test_generator.classes)

max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

In [ ]:
print(train_generator.class_indices)
print("Train: ", counter)
print("Valid: ", counter1)
print("Test: ", counter2)
print(class_weights)

{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}
Train:  Counter({0: 13192, 1: 6468, 2: 4444})
Valid:  Counter({0: 3298, 1: 1617, 2: 1111})
Test:  Counter({0: 200, 1: 100, 2: 100})
{0: 1.0, 1: 2.0395794681508965, 2: 2.9684968496849686}


# Create Model

In [ ]:
base_model = ResNet50V2(include_top=False, weights="imagenet", input_shape=(img_height, img_width, 3))

for layer in base_model.layers[:(len(base_model.layers) // 3 * 2)]:
    layer.trainable = False

for i, layer in enumerate(base_model.layers):
    print(i, layer.name, "-", layer.trainable)

94683136/94668760 [==============================] - 3s 0us/step
0 input_1 - False
1 conv1_pad - False
2 conv1_conv - False
3 pool1_pad - False
4 pool1_pool - False
5 conv2_block1_preact_bn - False
6 conv2_block1_preact_relu - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_pad - False
11 conv2_block1_2_conv - False
12 conv2_block1_2_bn - False
13 conv2_block1_2_relu - False
14 conv2_block1_0_conv - False
15 conv2_block1_3_conv - False
16 conv2_block1_out - False
17 conv2_block2_preact_bn - False
18 conv2_block2_preact_relu - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_pad - False
23 conv2_block2_2_conv - False
24 conv2_block2_2_bn - False
25 conv2_block2_2_relu - False
26 conv2_block2_3_conv - False
27 conv2_block2_out - False
28 conv2_block3_preact_bn - False
29 conv2_block3_preact_relu - False
30 conv2_block3_1_conv - False
31 conv2_block3_1_bn - Fal

In [ ]:
model = tf.keras.Sequential([
    base_model, 
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 15, 15, 2048)      23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 3

# Callbacks

In [ ]:
patience = 1
stop_patience = 3
factor = 0.5
callbacks = [
    ModelCheckpoint(os.path.join(ROOT_DIR, 'model/resnet.h5'), save_best_only=True, save_weights_only=True, verbose=1),
    # EarlyStopping(patience=stop_patience, monitor='val_loss', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience, verbose=1)
]

# Model Training

In [ ]:
epochs = 30
print(datetime.datetime.now())
history = model.fit(train_generator, validation_data=validation_generator, class_weight=class_weights, epochs=epochs, callbacks=callbacks, verbose=1)
print(datetime.datetime.now())

2022-09-21 05:17:29.381375
Epoch 1/30
754/754 [==============================] - ETA: 0s - loss: 0.4584 - accuracy: 0.9145 
Epoch 1: val_loss improved from inf to 0.20588, saving model to /content/drive/MyDrive/DTA/Detect-COVID19/model/resnet.h5
754/754 [==============================] - 11191s 15s/step - loss: 0.4584 - accuracy: 0.9145 - val_loss: 0.2059 - val_accuracy: 0.9326 - lr: 0.0010
Epoch 2/30
754/754 [==============================] - ETA: 0s - loss: 0.3173 - accuracy: 0.9459
Epoch 2: val_loss improved from 0.20588 to 0.13639, saving model to /content/drive/MyDrive/DTA/Detect-COVID19/model/resnet.h5
754/754 [==============================] - 1471s 2s/step - loss: 0.3173 - accuracy: 0.9459 - val_loss: 0.1364 - val_accuracy: 0.9494 - lr: 0.0010
Epoch 3/30
754/754 [==============================] - ETA: 0s - loss: 0.2813 - accuracy: 0.9518
Epoch 3: val_loss improved from 0.13639 to 0.11480, saving model to /content/drive/MyDrive/DTA/Detect-COVID19/model/resnet.h5
754/754 [=======

In [ ]:
with open(os.path.join(ROOT_DIR, 'history_dict_resnet.pkl'), 'wb') as file_pi:
    pickle.dump(history.history, file_pi)